# **한국어 형태소 분석**

### 1. 형태소 분석이란?

> 형태소(morpheme): 의미를 가지는 가장 작은 단위
> 
> 예:
> 
> - “재미있었다” → `재미 / 있 / 었 / 다`
> - “배송이 빠르네요” → `배송 / 이 / 빠르 / 네요`

|구분|영어(Word Tokenization)|한국어(형태소 분석)|
|---|---|---|
|단어 단위|공백 기준 분리 가능|조사, 어미 등 때문에 복잡|
|예시|“I love movie” → [I, love, movie]|“영화를 좋아한다” → [영화, 를, 좋아, 한다]|
|문제점|없음|“좋아”, “좋아요”, “좋았다” → 형태가 다름|

👉 따라서 한국어에서는 **형태소 분석기**를 이용해야 문장을 올바르게 나눌 수 있습니다.

---

### 🧩 2. Okt(Open Korean Text) 소개

|항목|설명|
|---|---|
|**이름**|Open Korean Text (구 Twitter 한국어 분석기)|
|**제공 모듈**|`konlpy.tag.Okt()`|
|**특징**|한국어 문장을 형태소 단위로 나누고, 품사 태깅 및 어간(stem) 추출 가능|
|**자주 사용하는 메서드**|`.morphs()`, `.pos()`, `.nouns()`|


> 유사한 하위호환  mecab

In [3]:
from konlpy.tag import Okt
okt = Okt()

sentence = "이 영화는 정말 재미있고 감동적이었어요!"
print("🔹 형태소:", okt.morphs(sentence))
print("🔹 명사만:", okt.nouns(sentence))
print("🔹 품사 태깅:", okt.pos(sentence))

🔹 형태소: ['이', '영화', '는', '정말', '재미있고', '감동', '적', '이었어요', '!']
🔹 명사만: ['이', '영화', '정말', '감동']
🔹 품사 태깅: [('이', 'Noun'), ('영화', 'Noun'), ('는', 'Josa'), ('정말', 'Noun'), ('재미있고', 'Adjective'), ('감동', 'Noun'), ('적', 'Suffix'), ('이었어요', 'Verb'), ('!', 'Punctuation')]


###  3. 불용어 제거 + 어간 추출

In [4]:
stopwords = ['은','는','이','가','을','를','에','의','와','과',
             '도','으로','로','에서','라','하다','있다','되다','이다','그','저','것']

def tokenize_korean(text):
    tokens = okt.morphs(str(text), stem=True)        # 어간추출(stem=True)
    tokens = [w for w in tokens if w not in stopwords and len(w) > 1]
    return ' '.join(tokens)

sample = "이 영화는 정말 재미있고 감동적이었어요!"
print("✅ 전처리 전:", sample)
print("✅ 전처리 후:", tokenize_korean(sample))

✅ 전처리 전: 이 영화는 정말 재미있고 감동적이었어요!
✅ 전처리 후: 영화 정말 재미있다 감동


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 샘플 데이터
data = [
    "이 영화 정말 재미있어요",
    "배우 연기가 최악이에요",
    "음악이 감동적이었어요",
    "스토리가 너무 지루했어요",
]

# 1️⃣ 형태소 분석 및 정제
tokenized = [tokenize_korean(t) for t in data]

# 2️⃣ TF-IDF 변환
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(tokenized)

print("✅ 단어 목록:", vectorizer.get_feature_names_out())
print("\\n✅ 변환된 TF-IDF 행렬:\\n", X_vec.toarray())

✅ 단어 목록: ['감동' '너무' '배우' '스토리' '연기' '영화' '음악' '이에요' '재미있다' '정말' '지루하다' '최악']
\n✅ 변환된 TF-IDF 행렬:\n [[0.         0.         0.         0.         0.         0.57735027
  0.         0.         0.57735027 0.57735027 0.         0.        ]
 [0.         0.         0.5        0.         0.5        0.
  0.         0.5        0.         0.         0.         0.5       ]
 [0.70710678 0.         0.         0.         0.         0.
  0.70710678 0.         0.         0.         0.         0.        ]
 [0.         0.57735027 0.         0.57735027 0.         0.
  0.         0.         0.         0.         0.57735027 0.        ]]
